In [13]:
import cv2

In [19]:
import cv2
import numpy as np

# Open webcam using DirectShow (important for Windows)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    print("❌ Camera not opened")
    exit()

# Drawing canvas
canvas = None

prev_x, prev_y = None, None

while True:
    ret, frame = cap.read()

    if not ret:
        print("❌ Camera not opened")
        continue   # don't break, just skip frame
    
    frame = cv2.flip(frame, 1)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == 27:  # ESC key
        break

    if canvas is None:
        canvas = np.zeros_like(frame)

    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Skin color range (works in normal lighting)
    lower_skin = np.array([0, 30, 60])
    upper_skin = np.array([20, 150, 255])

    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    

    # Morphology to clean noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)

        if area > 800:
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))

            # Draw tracking circle
            cv2.circle(frame, center, int(radius), (0, 255, 0), 2)

            if prev_x is None and prev_y is None:
                prev_x, prev_y = center
            else:
                cv2.line(canvas, (prev_x, prev_y), center, (0, 0, 255), 5)
                prev_x, prev_y = center
        else:
            prev_x, prev_y = None, None
    else:
        prev_x, prev_y = None, None

    # Merge drawing with live frame
    combined = cv2.add(frame, canvas)

    cv2.imshow("🎨 Air Drawing - Press ESC to Exit | C to Clear", combined)

    key = cv2.waitKey(1)

    if key == 27:  # ESC key
        break
    elif key == ord('c'):
        canvas = np.zeros_like(frame)

cap.release()
cv2.destroyAllWindows()

In [15]:
# import cv2
# import numpy as np

# # Open webcam using DirectShow (important for Windows)
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# if not cap.isOpened():
#     print("❌ Camera not opened")
#     exit()

# canvas = None
# prev_x, prev_y = None, None

# while True:
#     ret, frame = cap.read()

#     if not ret:
#         print("❌ Camera not opened")
#         continue

#     frame = cv2.flip(frame, 1)

#     if canvas is None:
#         canvas = np.zeros_like(frame)

#     # Convert to HSV
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#     # Skin color range
#     lower_skin = np.array([0, 30, 60])
#     upper_skin = np.array([20, 150, 255])

#     mask = cv2.inRange(hsv, lower_skin, upper_skin)

#     # Remove noise
#     kernel = np.ones((5, 5), np.uint8)
#     mask = cv2.erode(mask, kernel, iterations=1)
#     mask = cv2.dilate(mask, kernel, iterations=2)

#     # Find contours
#     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     shape_name = ""

#     if contours:
#         cnt = max(contours, key=cv2.contourArea)
#         area = cv2.contourArea(cnt)

#         if area > 800:

#             # Shape detection
#             peri = cv2.arcLength(cnt, True)
#             approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
#             vertices = len(approx)

#             if vertices == 3:
#                 shape_name = "Triangle"

#             elif vertices == 4:
#                 x, y, w, h = cv2.boundingRect(approx)
#                 aspect_ratio = float(w) / h
#                 if 0.95 <= aspect_ratio <= 1.05:
#                     shape_name = "Square"
#                 else:
#                     shape_name = "Rectangle"

#             elif vertices == 5:
#                 shape_name = "Pentagon"

#             elif vertices > 6:
#                 shape_name = "Circle"

#             else:
#                 shape_name = "Unknown"

#             # Draw contour
#             cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)

#             # Find center
#             (x, y), radius = cv2.minEnclosingCircle(cnt)
#             center = (int(x), int(y))

#             # Draw tracking circle
#             cv2.circle(frame, center, int(radius), (255, 0, 0), 2)

#             # Write shape name
#             cv2.putText(frame, shape_name, (center[0] - 50, center[1] - 20),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

#             # Air drawing
#             if prev_x is None and prev_y is None:
#                 prev_x, prev_y = center
#             else:
#                 cv2.line(canvas, (prev_x, prev_y), center, (0, 0, 255), 5)
#                 prev_x, prev_y = center

#         else:
#             prev_x, prev_y = None, None
#     else:
#         prev_x, prev_y = None, None

#     # Merge drawing with frame
#     combined = cv2.add(frame, canvas)

#     cv2.imshow("🎨 Air Drawing with Shape Detection | ESC to Exit | C to Clear", combined)

#     key = cv2.waitKey(1)

#     if key == 27:  # ESC
#         break
#     elif key == ord('c'):
#         canvas = np.zeros_like(frame)

# cap.release()
# cv2.destroyAllWindows()


In [16]:
# import cv2
# import numpy as np

# # Open webcam
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# if not cap.isOpened():
#     print("❌ Camera not opened")
#     exit()

# canvas = None
# prev_x, prev_y = None, None
# shape_name = ""
# drawing = False   # To know if user is drawing

# while True:
#     ret, frame = cap.read()

#     if not ret:
#         continue

#     frame = cv2.flip(frame, 1)

#     if canvas is None:
#         canvas = np.zeros_like(frame)

#     # Convert to HSV
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#     # Skin color range
#     lower_skin = np.array([0, 30, 60])
#     upper_skin = np.array([20, 150, 255])

#     mask = cv2.inRange(hsv, lower_skin, upper_skin)

#     # Remove noise
#     kernel = np.ones((5, 5), np.uint8)
#     mask = cv2.erode(mask, kernel, iterations=1)
#     mask = cv2.dilate(mask, kernel, iterations=2)

#     # Find contours (hand detection)
#     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if contours:
#         cnt = max(contours, key=cv2.contourArea)
#         area = cv2.contourArea(cnt)

#         if area > 800:
#             drawing = True
#             (x, y), radius = cv2.minEnclosingCircle(cnt)
#             center = (int(x), int(y))

#             if prev_x is None and prev_y is None:
#                 prev_x, prev_y = center
#             else:
#                 cv2.line(canvas, (prev_x, prev_y), center, (0, 0, 255), 5)
#                 prev_x, prev_y = center
#         else:
#             prev_x, prev_y = None, None
#     else:
#         # When hand disappears → detect shape
#         if drawing:
#             drawing = False
#             prev_x, prev_y = None, None

#             # Convert canvas to grayscale
#             gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
#             _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

#             contours_drawn, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             if contours_drawn:
#                 cnt_drawn = max(contours_drawn, key=cv2.contourArea)
#                 area_drawn = cv2.contourArea(cnt_drawn)

#                 if area_drawn > 1000:
#                     peri = cv2.arcLength(cnt_drawn, True)
#                     approx = cv2.approxPolyDP(cnt_drawn, 0.02 * peri, True)
#                     vertices = len(approx)

#                     if vertices == 3:
#                         shape_name = "Triangle"

#                     elif vertices == 4:
#                         x, y, w, h = cv2.boundingRect(approx)
#                         aspect_ratio = float(w) / h
#                         if 0.95 <= aspect_ratio <= 1.05:
#                             shape_name = "Square"
#                         else:
#                             shape_name = "Rectangle"

#                     elif vertices == 5:
#                         shape_name = "Pentagon"

#                     elif vertices > 6:
#                         shape_name = "Circle"

#                     else:
#                         shape_name = "Unknown"

#     # Merge frame and canvas
#     combined = cv2.add(frame, canvas)

#     # Display shape name after drawing
#     if shape_name != "":
#         cv2.putText(combined, "Shape: " + shape_name, (20, 50),
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

#     cv2.imshow("🎨 Air Drawing - Shape After Drawing | ESC Exit | C Clear", combined)

#     key = cv2.waitKey(1)

#     if key == 27:  # ESC
#         break
#     elif key == ord('c'):
#         canvas = np.zeros_like(frame)
#         shape_name = ""

# cap.release()
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import numpy as np

# # Open webcam
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# if not cap.isOpened():
#     print("❌ Camera not opened")
#     exit()

# canvas = None
# prev_x, prev_y = None, None
# shape_name = ""
# drawing = False

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         continue

#     frame = cv2.flip(frame, 1)

#     if canvas is None:
#         canvas = np.zeros_like(frame)

#     # Convert to HSV
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#     # 🔵 Blue color HSV range
#     lower_blue = np.array([100, 150, 50])
#     upper_blue = np.array([140, 255, 255])

#     mask = cv2.inRange(hsv, lower_blue, upper_blue)

#     # Remove noise
#     kernel = np.ones((5, 5), np.uint8)
#     mask = cv2.erode(mask, kernel, iterations=1)
#     mask = cv2.dilate(mask, kernel, iterations=2)

#     # Find contours (blue object detection)
#     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     if contours:
#         cnt = max(contours, key=cv2.contourArea)
#         area = cv2.contourArea(cnt)

#         if area > 800:
#             drawing = True
#             (x, y), radius = cv2.minEnclosingCircle(cnt)
#             center = (int(x), int(y))

#             # Draw tracking circle
#             cv2.circle(frame, center, int(radius), (255, 0, 0), 2)

#             if prev_x is None:
#                 prev_x, prev_y = center
#             else:
#                 cv2.line(canvas, (prev_x, prev_y), center, (0, 0, 255), 5)
#                 prev_x, prev_y = center
#         else:
#             prev_x, prev_y = None, None
#     else:
#         # When blue object disappears → detect shape
#         if drawing:
#             drawing = False
#             prev_x, prev_y = None, None

#             gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
#             _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

#             contours_drawn, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#             if contours_drawn:
#                 cnt_drawn = max(contours_drawn, key=cv2.contourArea)
#                 area_drawn = cv2.contourArea(cnt_drawn)

#                 if area_drawn > 1000:
#                     peri = cv2.arcLength(cnt_drawn, True)
#                     approx = cv2.approxPolyDP(cnt_drawn, 0.02 * peri, True)
#                     vertices = len(approx)

#                     if vertices == 3:
#                         shape_name = "Triangle"

#                     elif vertices == 4:
#                         x, y, w, h = cv2.boundingRect(approx)
#                         aspect_ratio = float(w) / h
#                         if 0.95 <= aspect_ratio <= 1.05:
#                             shape_name = "Square"
#                         else:
#                             shape_name = "Rectangle"

#                     elif vertices == 5:
#                         shape_name = "Pentagon"

#                     elif vertices > 6:
#                         shape_name = "Circle"

#                     else:
#                         shape_name = "Unknown"

#     # Merge frame and drawing
#     combined = cv2.add(frame, canvas)

#     # Show detected shape
#     if shape_name != "":
#         cv2.putText(combined, "Shape: " + shape_name, (20, 50),
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

#     cv2.imshow("🔵 Blue Air Drawing | ESC Exit | C Clear", combined)

#     key = cv2.waitKey(1)

#     if key == 27:
#         break
#     elif key == ord('c'):
#         canvas = np.zeros_like(frame)
#         shape_name = ""

# cap.release()
# cv2.destroyAllWindows()
